# Matching the WEST ICRH Antenna

In this notebook we investigate the various method to match a WEST ICRH antenna. By matching the antenna we mean to find 4 capacitances values $C_1$, $C_2$, $C_3$ and $C_4$ in order for the antenna to be operated at a given frequency $f_0$. 

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
import skrf as rf
import numpy as np
import sys
from tqdm.notebook import tqdm

sys.path.append('..')
from west_ic_antenna import WestIcrhAntenna

In [ ]:
rf.stylely()

In [ ]:
from west_ic_antenna import WestIcrhAntenna

## Matching Each Sides Separately
Here, each side of the antenna is matched separatly, which leads to two set of capacitances $(C_1, C_2)$ and $(C_3,C_4)$.

In the following example, both sides of the antenna are matched at the frequency $f_0$, keeping opposite side unmatched (C=150pF):

In [ ]:
f0 = 55e6
freq = rf.Frequency(54, 56, npoints=1001, unit='MHz')
ant = WestIcrhAntenna(frequency=freq)  # default is vacuum coupling

In [ ]:
C_match_left = ant.match_one_side(f_match=f0, side='left', solution_number=1)
C_match_right = ant.match_one_side(f_match=f0, side='right', solution_number=1)
print('Left side matching point: ', C_match_left)
print('Right side matching point: ', C_match_right)

Let's have a look to the RF reflection coefficient of each sides:

In [ ]:
fig, ax = plt.subplots()
ant.circuit(Cs=C_match_left).network.plot_s_db(m=0, n=0, lw=2, ax=ax)
ant.circuit(Cs=C_match_right).network.plot_s_db(m=1, n=1, lw=2, ls='--', ax=ax)
ax.legend(('Left side matched (right unmatched)', 
           'Right side matched (left side unmatched)'))

In reality, the precision at which one can tune the capacitance is not better than 1/100 pF, so one have to consider rounding optimal solutions to such precision :


In [ ]:
C_match_left = ant.match_one_side(f_match=f0, side='left', solution_number=1, decimals=2)
C_match_right = ant.match_one_side(f_match=f0, side='right', solution_number=1, decimals=2)
print('Left side matching point: ', C_match_left)
print('Right side matching point: ', C_match_right)

Note that the performances are slightly degraded, but, it's real life! 

In [ ]:
fig, ax = plt.subplots()
ant.circuit(Cs=C_match_left).network.plot_s_db(m=0, n=0, lw=2, ax=ax)
ant.circuit(Cs=C_match_right).network.plot_s_db(m=1, n=1, lw=2, ls='--', ax=ax)
ax.legend(('Left side matched (right unmatched)', 
           'Right side matched (left side unmatched)'))
ax.set_ylabel('$|S_{ii}|$ [dB]')
ax.set_xlabel('f [MHz]')
ax.legend(('$S_{11}$', '$S_{22}$'))

## Frequency Shift for Dipole Excitation
The coupling between antenna's sides requires shifting the frequency with respect to the matching frequency used for each side separately.

Thus, if the optimal capacitor set for both side are of the same kind (i.e. either both C_top>C_bot or both C_top<C_bot), then dipole excitation (phase $(0,\pi)$) requires to shift the frequency to higher frequency to operate the antenna in a optimal conditions:

In [ ]:
# dipole excitation
power = [1, 1]
phase = [0, rf.pi]
# combine both separate solutions
C_match = [C_match_left[0], C_match_left[1], C_match_right[2], C_match_right[3]]
# looking to the active s parameters:
s_act = ant.s_act(power, phase, Cs=C_match)
# finding the optimum frequency
idx_f_opt = np.argmin(np.abs(s_act[:,0]))
f_opt = freq.f[idx_f_opt]
delta_f = f_opt - f0
print(f'Optimum frequency is f_opt={f_opt/1e6} MHz --> {delta_f/1e6} MHz shift' )

In [ ]:
fig, ax = plt.subplots()
ax.plot(freq.f_scaled, 20*np.log10(np.abs(s_act)), lw=2)
ax.axvline(f0/1e6, ls='--', color='gray')
ax.axvline(f_opt/1e6, ls='--', color='k')
ax.set_ylabel('$|S_{act}|$ [dB]')
ax.set_xlabel('f [MHz]')
ax.legend(('$S_{act,1}$', '$S_{act,2}$'))
ax.set_title('Dipole excitation')

Monopole excitation (phase $(0,0)$) at the contrary requires shifting the operating frequency toward lower frequencies:

In [ ]:
# monopole excitation
power = [1, 1]
phase = [0, 0]
# combine both separate solutions
C_match = [C_match_left[0], C_match_left[1], C_match_right[2], C_match_right[3]]
# looking to the active s parameters:
s_act = ant.s_act(power, phase, Cs=C_match)
# finding the optimum frequency
idx_f_opt = np.argmin(np.abs(s_act[:,0]))
f_opt = freq.f[idx_f_opt]
delta_f = f_opt - f0
print(f'Optimum frequency is f_opt={f_opt/1e6} MHz --> {delta_f/1e6} MHz shift' )

In [ ]:
fig, ax = plt.subplots()
ax.plot(freq.f_scaled, 20*np.log10(np.abs(s_act)), lw=2)
ax.axvline(f0/1e6, ls='--', color='gray')
ax.axvline(f_opt/1e6, ls='--', color='k')
ax.set_ylabel('$|S_{act}|$ [dB]')
ax.set_xlabel('f [MHz]')
ax.legend(('$S_{act,1}$', '$S_{act,2}$'))
ax.set_title('Monopole excitation')

## Matching Both Sides at the same time
It is also possible to optimize the antenna directly for the target frequency and for a given excitation. Matching both sides at the same time in fact matches each side separately, then find the optimum points using these solutions as starting point to help the convergence of the optimization. 

In [ ]:
f0 = 55e6
freq = rf.Frequency(54, 56, npoints=1001, unit='MHz')
ant = WestIcrhAntenna(frequency=freq)  # default is vacuum coupling

# antenna excitation to match for
power = [1, 1]  # W
phase = [0, np.pi]  # dipole

ant.DEBUG=True  # display additional messages
# Providing an initial guess C0 skip the search on both sides separately
C0 = [C_match_left[0], C_match_left[1], C_match_right[2], C_match_right[3]]
Cs = ant.match_both_sides(f_match=f0, power=power, phase=phase, C0=C0, decimals=2)

In [ ]:
# looking to the active s parameters:
s_act = ant.s_act(power, phase, Cs=Cs)

In [ ]:
fig, ax = plt.subplots()
ax.plot(freq.f_scaled, 20*np.log10(np.abs(s_act)), lw=2)
ax.axvline(f0/1e6, ls='--', color='gray')
ax.set_ylabel('$|S_{act}|$ [dB]')
ax.set_xlabel('f [MHz]')
ax.legend(('$S_{act,1}$', '$S_{act,2}$'))

# Manual Matching on Plasma
When the antenna is facing the plasma, the coupling resistance increases and the antenna matching is affected.

In [ ]:
C_vacuum = Cs  # previous dipole set point in vacuum for f0

# changing the front-face of the antenna to a plasma case
freq = rf.Frequency(54, 56, npoints=1001, unit='MHz')
ant = WestIcrhAntenna(frequency=freq,
                     front_face='../west_ic_antenna/data/Sparameters/front_faces/TOPICA/S_TSproto12_55MHz_Hmode_LAD6-2.5cm.s4p')  

# looking to the active s parameters in dipole:
powers = [1 ,1]
phases = [0, np.pi] 
s_act = ant.s_act(powers, phases , Cs=Cs)

fig, ax = plt.subplots()
ax.plot(freq.f_scaled, 20*np.log10(np.abs(s_act)), lw=2)
ax.axvline(f0/1e6, ls='--', color='gray')
ax.set_ylabel('$|S_{act}|$ [dB]')
ax.set_xlabel('f [MHz]')
ax.legend(('$S_{act,1}$', '$S_{act,2}$'))
ax.set_title('Reflection on plasma using vacuum matching setpoint')

Thus, it is necessary to adapt the capacitors to improve the matching of the antenna

In [ ]:
C_match_plasma = ant.match_both_sides(f0, power=powers, phase=phases, C0=C_vacuum)

In [ ]:
s_act = ant.s_act(powers, phases , Cs=C_match_plasma)

fig, ax = plt.subplots()
ax.plot(freq.f_scaled, 20*np.log10(np.abs(s_act)), lw=2)
ax.axvline(f0/1e6, ls='--', color='gray')
ax.set_ylabel('$|S_{act}|$ [dB]')
ax.set_xlabel('f [MHz]')
ax.legend(('$S_{act,1}$', '$S_{act,2}$'))
ax.set_title('Reflection on plasma after rematching')

If we compare the capacitor setpoint for vacuum and plasma, we see that the difference between the value is approximatively constant:
* top capacitances are increased 
* bottom capacitances are decreased

In [ ]:
np.array(C_match_plasma) - np.array(C_vacuum)

The capacitance shift to apply with respect to the vacuum setpoint depends of the coupling resistance of the plasma.

We can use this property to deduce the set of capacitors to use during plasma operation. Below we generate a series of various plasma loading cases, of increasing coupling resistance $R_c$. For each case we search for and we store the setpoints.

In [ ]:
Rcs = np.linspace(1, 2.9, 10)
C_matchs = []
for Rc in tqdm(Rcs):
    _plasma = WestIcrhAntenna.TOPICA_front_face(Rc=Rc, mode='L')
    _ant = WestIcrhAntenna(frequency=freq, front_face=_plasma)
    _C_match = _ant.match_both_sides(f0, power=powers, phase=phases, C0=C_vacuum, delta_C=10)
    C_matchs.append(_C_match)

Let's plot the average capacitance shift to apply versus the coupling resistance of the plasma loading scenario:

In [ ]:
delta_C_avg = np.mean(np.abs(np.array(C_matchs) - np.array(C_vacuum)), axis=1)
delta_C_std = np.std(np.abs(np.array(C_matchs) - np.array(C_vacuum)), axis=1)

In [ ]:
fig, ax = plt.subplots()
ax.fill_between(Rcs, delta_C_avg-delta_C_std, delta_C_avg+delta_C_std, alpha=0.3)
ax.plot(Rcs, np.abs(np.array(C_matchs)-np.array(C_vacuum)))
ax.plot(Rcs, delta_C_avg, ls='--', color='k', lw=2)
ax.set_xlabel('Coupling Resistance $R_c$ [Ohm]')
ax.set_ylabel('Average Capacitance Shift $\Delta C$ [pF]')
ax.set_title('Capacitance Shift to add from Vacuum Match Points')
ax.set_ylim(top=10)

This plot should be benchmark against plasma measurements.

In [ ]:
from IPython.core.display import HTML
def _set_css_style(css_file_path):
    """
    Read the custom CSS file and load it into Jupyter
    Pass the file path to the CSS file
    """
    styles = open(css_file_path, "r").read()
    s = '<style>%s</style>' % styles
    return HTML(s)

_set_css_style('custom.css')